### Create phylogenetic trees
- Ran Genomad, 3,498 hits with RdRps. 
- Also gives taxonomy
- Downloaded NCBI RefSeq (v. 224) RdRp sequences (n=2124)


TODO:
- deduplicate the RdRps at 99% AAI
- For each RdRp, align with proper taxonomic affiliation
- Align with all RdRps from NCBI. 


Start with the Tombusviridae, cause only 4 hits. +70 refseq (this worked nicely)


In [ ]:
# Deduplicate the RdRps of interest (4642 sequences left, great!):
mamba activate cdhit
cd-hit -i  own_refseq.rdrp.faa -o  own_refseq.rdrp.dedup.faa \
-c 0.99 -T 1 -d 0

In [ ]:
# Create an overall RdRp tree
srun --account=ctbrowngrp -p bmh -J tree_build -t 12:00:00 -c 24 --mem=50gb --pty bash

mamba activate clustalo
clustalo -i own_refseq.rdrp.dedup.faa \
--auto -t Protein --threads=30 \
-o allrdrp.clustalo.faa

In [ ]:
# Use snakemake to make family specific trees
snakemake --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 24 --rerun-incomplete -k -n

In [ ]:
# trimal 
srun --account=ctbrowngrp -p bmh -J tree_build -t 1:00:00 -c 1 --mem=20gb --pty bash
mamba activate trimal 

# do trimal
trimal -in allrdrp.clustalo.faa \
-out allrdrp.trimal.faa -gappyout


# fasttree
mamba activate fasttree
FastTree < allrdrp.trimal.faa > allrdrp.fasttree.nwk \
-wag -log all.fasttree.log 

In [ ]:
# Fasttree
srun --account=ctbrowngrp -p bmm -J fastree -t 5:00:00 -c 2 --mem=50gb --pty bash

# Use FastTree to make phylogenetic trees
mamba activate fasttree
FastTree < alignrdrp.clustalo_trimal.faa > rdrd.align.all.fasttree.nwk \
-wag -log all.fasttree.log 

# Family trees:
- BlastP (diamond) agianst refseq
- anything that has a hit gets in a tree with that thing
- make fam level trees
- crosscheck with GEnomad


In [ ]:
mamba activate seqkit
seqkit grep -n -f head.txt own_refseq.rdrp.faa -o own.dedup.faa

In [ ]:
# create a diamond db 
mamba activate diamond
diamond makedb --in refseqrdrp.ns.faa --db refseqrdrp.ns.dmnd

# do blastp (out of 2656, 2028 aligned)
mamba activate diamond
diamond blastp -q own.dedup.faa \
--max-target-seqs 1 --header \
-e 1 --threads 1 \
--very-sensitive \
--db refseqrdrp.ns.dmnd \
-o refseq.genomad.blastp.tsv